In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Import data

In [2]:
#btc options data
df=pd.read_csv('data/btc_data_bsm.csv', sep=",", index_col=0)
df = df.reset_index(drop=False)
#df.drop(columns=['Unnamed: 0', 'interest_rate'], inplace=True)

In [1]:
import requests
import pandas as pd
from datetime import datetime

# Alpha Vantage API configuration
API_KEY = 'NW05RFO7K02SAMAL'
SYMBOL = 'BTC'
MARKET = 'USD'

def fetch_historical_data(symbol, market, api_key):
    url = f'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol={symbol}&market={market}&apikey={api_key}'
    response = requests.get(url)
    data = response.json()
    time_series = data.get('Time Series (Digital Currency Daily)', {})
    
    # Convert to DataFrame
    df = pd.DataFrame.from_dict(time_series, orient='index')
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    
    # Rename columns for convenience
    df.rename(columns={
        '1a. open (USD)': 'open',
        '2a. high (USD)': 'high',
        '3a. low (USD)': 'low',
        '4a. close (USD)': 'close',
        '5. volume': 'volume',
        '6. market cap (USD)': 'market_cap'
    }, inplace=True)
    
    # Convert to numeric
    df = df.apply(pd.to_numeric)
    return df

# Fetch the data
historical_data = fetch_historical_data(SYMBOL, MARKET, API_KEY)
print(historical_data.head())


             1. open   2. high    3. low  4. close        volume
2023-07-20  29916.69  30421.29  29564.19  29809.13  11494.953488
2023-07-21  29809.13  30060.28  29729.48  29907.98   7597.810267
2023-07-22  29907.15  30002.38  29626.86  29795.03   3629.482511
2023-07-23  29793.62  30350.70  29733.55  30081.61   4513.832143
2023-07-24  30081.61  30099.99  28850.00  29176.98  14484.275440
